# RDkit: Open-Source cheminformatics

RDKit documentation:
- https://github.com/rdkit
- https://www.rdkit.org/
- https://www.rdkit.org/docs/Cookbook.html


RDKit is an open-source cheminformatics toolkit widely used in computational chemistry and drug discovery. It provides a comprehensive set of tools and libraries for working with chemical structures. Some of the most common libraries within RDKit include:

    Chem: The core library for working with chemical structures, enabling the creation, modification, and analysis of molecules. It allows for tasks like substructure searching, molecular property calculations, and structure rendering.

    ChemDraw: A library for generating 2D chemical structure diagrams and visual representations. It is often used for drawing and displaying chemical structures in scientific publications.

    AllChem: This library focuses on the handling of 3D chemical structures. It provides tools for 3D molecular conformer generation, alignment, and structure optimization, essential for tasks like virtual screening and molecular docking.

    Descriptors: This library is used to calculate molecular descriptors, which are numerical values that represent various chemical properties and features of molecules. Descriptors are crucial for quantitative structure-activity relationship (QSAR) modeling.

In [267]:
# Import necessary libraries
from rdkit import Chem  # RDKit library for working with molecules
from rdkit.Chem import Draw, AllChem, MACCSkeys  # Submodules for molecule visualization, fingerprints, and shape comparison
import pandas as pd  # Library for data manipulation and analysis
from rdkit import DataStructs # Import module for working with fingerprint similarity
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

## Query molecule: Ibuprofen

We could depict molecules using diferents formats:
- Smiles: MolFromSmiles
- Smarts: MolFromSmarts
- Mol file information: MolFromMolFile

*Depict one molecule with its Smiles*
    
 CC(C)CC1=CC=C(C=C1)C(C)C(=O)O

In [180]:
# Load a molecule from SMILES
# smiles = 'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O'
smiles = 'CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21'
mol = Chem.MolFromSmiles(smiles) # convert the SMILES string into an RDKit molecule object

In [ ]:
# Draw the molecule
Draw.MolToImage(mol)

In [ ]:
#Also we could print mol file information
print(Chem.MolToMolBlock(mol))

In [ ]:
# Add Hydrogens:
QueryH = Chem.AddHs(mol)
Draw.MolToImage(QueryH)

In [ ]:
# Remove Hydrogens
QuerynoH = Chem.RemoveHs (QueryH)
Draw.MolToImage(QuerynoH)

In [185]:
# Write Smiles
w1 = Chem.SmilesWriter('brinzolamide.smi')
w1.write(QueryH)
w1.close()

In [186]:
# Write molecules in sdf format:
w2 = Chem.SDWriter('brinzolamide.sdf')
w2.write(QueryH)
w2.close()

In [ ]:
# Generate 3D coordinates for the molecule
AllChem.EmbedMolecule(QueryH, randomSeed=42)  # You can provide a random seed for reproducibility
# Optimize the 3D coordinates
AllChem.MMFFOptimizeMolecule(QueryH, mmffVariant="MMFF94")

EmbedMolecule and MMFFOOptimizeMolecule functions from RDKit's AllChem module do not return a molecule object with 3D coordinates. Instead, it modifies the provided molecule object in place by adding 3D coordinates to it.

In [ ]:
Draw.MolToImage(QueryH)

In [ ]:
print(Chem.MolToMolBlock(QueryH))

In [190]:
# Write molecules in sdf format:
w3 = Chem.SDWriter('optimizeexample.sdf')
w3.write(QueryH)
w3.close()

In [191]:
# Create an SDWriter object
w3 = Chem.SDWriter('optimizeexamplewithcomments.sdf')

# Set a comment for the molecule
comment = "This is a comment for the molecule."

# Add the comment to the specific data field
mol.SetProp("COMMENT", comment)

# Save the molecule to the SDF file
w3.write(mol)
w3.close()

In [218]:
r = requests.get('https://www.ebi.ac.uk/chembl/api/data/similarity/CHEMBL220491/50.json')
hits = pd.DataFrame.from_records([dict(smiles=m['molecule_structures']['canonical_smiles'], similarity=float(m['similarity']), chembl_id=m['molecule_chembl_id']) for m in r.json()['molecules']])
hits.query('similarity < 90').sample(9)[['chembl_id', 'smiles']].to_csv('hits.csv')

#  Virtual Screening
### Fingerprint Generation
1) MACCS keys (https://github.com/rdkit/rdkit/blob/master/rdkit/Chem/MACCSkeys.py: Molecular ACCess System (MACCS) fingerprints, also termed MACCS structural keys, consist of 166 predefined structural fragments. Each position queries the presence or absence of one particular structural fragment or key.

2) Morgan Fingerprints (Circular fingerprints) ( (https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints): This family of fingerprints is based on the Morgan algorithm. The bits correspond to the circular environments of each atom in a molecule. The number of neighboring bonds and atoms to consider is set by the radius. Also the length of the bit string can be defined, a longer bit string will be modded to the desired length. Therefore, the Morgan fingerprint is not limited to a certain number of bits.


Getting Started with the RDKit in Python
(https://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity)

<u><strong> MACCS Keys

In [219]:
MACCQuery = MACCSkeys.GenMACCSKeys(mol)

In [ ]:
MACCQuery.ToBitString()   

Now, we use a dataset of ~ 400 molecules to perform ligand-based screening.

In [221]:
dataset = pd.read_csv('example-dataset.csv', sep=";", header=None, names=['chembl_id', 'smiles'])

In [ ]:
num_compounds = len(dataset)
num_compounds

If you need to fetch a large number of compounds, you might need to implement pagination by making multiple requests with smaller limits and then combining the results. This way, you can ensure that you're not overwhelming the API and are able to retrieve the data you need.

In [ ]:
# Convert SMILES into rdkit objects
dataset['rdkitmol'] = dataset['smiles'].apply(Chem.MolFromSmiles)
# Calculate MACCS fingerprints for the query molecule and fetched molecules
dataset['MACCS_fps'] = dataset['rdkitmol'].apply(MACCSkeys.GenMACCSKeys)
# Calculate Tanimoto similarity between the query fingerprint and fetched fingerprints
dataset['MACCS_ts'] = dataset['MACCS_fps'].apply(lambda x: DataStructs.TanimotoSimilarity(MACCQuery, x))
# Display the top 10 hits
dataset.sort_values(by='MACCS_ts', ascending=False)[:10]

In [ ]:
sns.relplot(x=np.arange(len(dataset)), y=dataset['MACCS_ts'].sort_values(ascending=False), aspect=2.0, height=2.0, kind='line')

**Small exercise 1** Can you connect the hit compounds with their names using a REST-API?

**Small exercise 2** Generate your own dataset and run it through the pipeline.

<u><strong> Morgan fingerprints

We can also calculate the circular Morgan fingerprints with rdkit. The Morgan fingerprint can be calculated either as int or bit vector. By default the radius is 2 and the vector is 2048 long.

In [ ]:
MorganQuery = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)

In [ ]:
MorganQuery.ToBitString()

Applied to all molecules: Add Morgan fingerprints for all molecules to the DataFrame

In [ ]:
#Calculate MACCS fingerprints for all molecules in the dataset
dataset['Morgan_fps'] = dataset['rdkitmol'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048))

In [ ]:
# Calculate Tanimoto similarity between the query molecule and dataset molecules
# Sort dataset by similarity and print top similar molecules
dataset['Morgan_ts'] = dataset['Morgan_fps'].apply(lambda x: DataStructs.FingerprintSimilarity(MorganQuery, x))

dataset.sort_values(by='Morgan_ts', ascending=False)[['chembl_id', 'smiles', 'Morgan_ts']][:10]

In [ ]:
sns.relplot(x=np.arange(len(dataset)), y=dataset['Morgan_ts'].sort_values(ascending=False), aspect=2.0, height=2.0, kind='line')

**Exercise 3** Do Morgan and MACCS fingerprints correlate?

In [ ]:
sns.relplot(x='Morgan_ts', y='MACCS_ts', data=dataset)

## Shape-based Virtual Screening

In shape-based virtual screening, the goal is to compare the 3D shape and spatial arrangement of molecules to identify compounds with similar shapes. This approach is particularly useful when the exact chemical structure and interactions are not well-defined, but the overall shape is an important factor for biological activity. For example, when dealing with receptor-based binding or protein-ligand interactions, shape complementarity is a critical factor. So, the first step is to obtain the 3D coordinates


**Hydrogen Addition**: Prior to 3D structure generation, add explicit hydrogen atoms to each molecule. This step is vital as hydrogen atoms play a crucial role in intermolecular interactions, particularly hydrogen bonding.

**3D Structure Generation**: Generate 3D structures for each molecule in your dataset. This step is vital as it allows for the accurate representation of the molecular shape in three dimensions. 

**Rotamer Calculation**: If dealing with flexible molecules, consider calculating rotamers to explore the conformational space. 

**Molecular Alignment**: Align all molecules to a reference molecule or a template, often the bioactive ligand or a known active compound. Molecular alignment is crucial for superimposing and comparing the shapes of molecules.

**Shape-Based Screening**: Compare the shape descriptors of the aligned molecules with those of the reference molecule. Identify molecules with shape descriptors that closely match the reference as potential ligands. Various similarity metrics like Tanimoto coefficients can be applied.

**Ranking and Selection**: Rank the screened molecules based on their shape similarity to the reference molecule. Select the top-ranking molecules as potential ligands.

***RDKit Aligner method: Open3DALIGN object***

The methodology behind the O3A alignment is a technique used to align two molecules in 3D space while minimizing the differences in their atomic positions. 
It calculates a weighted RMSD (root-mean-square deviation) between the atom coordinates of the two molecules. The algorithm iteratively refines the alignment by minimizing the RMSD. The RMSD value quantifies the dissimilarity between the aligned structures. A lower RMSD indicates a better alignment, where the structures are more similar in terms of their 3D spatial arrangement. When the RMSD is nearly zero, it means that the molecules are almost identical in their 3D conformation.

o3a = AllChem.GetO3A(query_conformer, target_molecule, query_conformer, target_conformer)
rmsd = o3a.Align()  # Calculate and get the RMSD from the alignment

<u>Arguments:<u>

    query_conformer: This is the conformer (3D structure) of the query molecule that you want to overlay onto the target molecule. 
    target_molecule: This is the target molecule to which you want to overlay the query conformer. 
    query_conformer: This argument should be the same as the first one. It allows you to use the same conformer for both query and target for various alignment options, but it's often set to the same value as query_conformer.
    target_conformer: This is the conformer of the target molecule.

<u>Alignment Process:<u>

    Superimposes and aligns the query conformer onto the target conformer based on their 3D coordinates. The alignment is typically done by minimizing the RMSD between the two conformers. This step involves the optimization of the alignment to find the best fit.
    
        1) Superposition: The first step in calculating the RMSD is to align or superimpose the two molecular conformations to be compared. This superposition process involves translating and rotating one of the molecules to minimize their spatial differences.
        2) Distance Calculation: After superposition, the corresponding atoms of the two molecules are compared. The RMSD is calculated as the square root of the average of the squared differences in the positions of these atoms.
        3) Interatomic Distances: The RMSD considers all atom pairs that correspond to one another in the aligned structures. It essentially measures the deviation of atom positions between the two molecules. Smaller RMSD values indicate a higher degree of 3D similarity or overlap.

    Result Interpretation: A low RMSD value implies that the two molecules have a high level of similarity in their 3D shape. Conversely, a high RMSD value indicates substantial dissimilarity.

<u>Align two molecules<u>

In [231]:
# Define two example molecules
mol1 = Chem.MolFromSmiles('CCO')
mol2 = Chem.MolFromSmiles('CCC=O')

In [ ]:
mol1

In [ ]:
mol2

In [ ]:
# Generate 3D conformations for mol1
mol1 = Chem.AddHs(mol1)
AllChem.EmbedMolecule(mol1, randomSeed=42)
AllChem.MMFFOptimizeMolecule(mol1)
mol1

In [ ]:
# Generate 3D conformations for mol2
mol2 = Chem.AddHs(mol2)
AllChem.EmbedMolecule(mol2, randomSeed=42)
AllChem.MMFFOptimizeMolecule(mol2)
mol2

In [ ]:
# Now we are going to generate 25 conformations for mol2
AllChem.EmbedMultipleConfs(mol2, numConfs=25)
AllChem.MMFFOptimizeMoleculeConfs(mol2)

In [ ]:
# Initialize the best alignment score
best_alignment_score = float('inf')

# Perform alignment and calculate the score for each conformer of mol2 with respect to mol1
for cid in range(mol2.GetNumConformers()):
    o3a = AllChem.GetO3A(mol2, mol1, prbCid=cid)
    o3a.Align()
    alignment_score = o3a.Align()

    # Update the best alignment score if a lower value is found
    if alignment_score < best_alignment_score:
        best_alignment_score = alignment_score

print("The best alignment score is:", best_alignment_score)

*Generate conformers
AllChem.EmbedMultipleConfs: This is a function from the RDKit library that generates multiple conformers of a molecule. It uses a combination of random sampling, energy minimization, and scoring to generate multiple conformers for a molecule (different spatial arrangements). 

    mol: This is the molecule for which you want to generate conformers. It should be an RDKit molecule object.

    numConfs=num_conformers: This argument specifies how many conformers you want to generate, and it takes the value of num_conformers we set earlier.
    
    params=AllChem.ETKDG: Including energy minimization during the conformer generation can help generate more physically realistic conformations. Instance of the ETKDG (Extended Tight-Binding Quantum Chemical Molecular Dynamics) parameters for conformer generation in RDKit. ETKDG is an enhanced conformer generation method that uses a knowledge-based approach to generate reasonable 3D structures of molecules.

The generated conformers will be stored in the conformers variable as a list of RDKit molecule objects. 

Generating conformers is a multi-step process that involves several operations to create multiple spatial arrangements of the molecule. 

**Distance Geometry Embedding**: Distance geometry embedding, where the initial spatial positions of the atoms are randomly assigned. This generates an initial, random 3D conformation for the molecule. The goal is to find conformations where the interatomic distances match the ideal distances as closely as possible.

**Energy Minimization (optional)**: An energy minimization step can be included to optimize the initial conformations. This step adjusts the atom positions to minimize the total energy of the system, making the conformer more stable.

**Random Perturbations**: To explore different conformations, the conformer generator introduces random perturbations to the positions of atoms. These perturbations are applied to dihedral angles, bond lengths, and bond angles, allowing the molecule to sample a broader conformational space.

**Distance Constraints**: The generator applies distance constraints to ensure that bonds between atoms don't become too short or too long. These constraints help maintain realistic bond lengths during the conformational search.

**Clash Detection**: The conformer generator checks for clashes or overlaps between atoms in the conformation. If overlaps occur, it makes adjustments to eliminate the clashes.

**Energy Scoring**: For each generated conformation, an energy score is computed. This score quantifies how well the conformation corresponds to the lowest-energy state of the molecule. The goal is to find conformations with low-energy scores, indicating more stable conformations.

**Selection of Best Conformers**: Finally, multiple conformations are generated, and the best ones are selected based on their energy scores. You can specify how many conformers you want to generate (as in num_conformers), and the function returns the top-scoring conformers as a list.

<u>Align Dataset: multiple conformers<u>

In [ ]:
# Generate 3D coordinates for the molecule
query_smiles = 'CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21'
query = Chem.MolFromSmiles(query_smiles)
query_h = Chem.AddHs(query)
AllChem.EmbedMolecule(query_h, randomSeed=42)  # You can provide a random seed for reproducibility
# Optimize the 3D coordinates
AllChem.MMFFOptimizeMolecule(query_h, mmffVariant="MMFF94")

In [ ]:
O3A = AllChem.GetO3A(query_h, query_h, prbCid=0)  # Get an O3A object for superposition
O3A.Align()

In [264]:
def embed_optimize_align_molecule(mol, query, num_conformers = 2):
  """
  This function takes ´mol´ rdkit object, embeds an initial geometry, optimizes such a geometry
  to generate a set of n confomers, and then it aligns structurally against the reference.

  It returns ´None´ if the molecule optimization was unsuccesful
  """
  mol = Chem.AddHs(mol)
  AllChem.EmbedMultipleConfs(mol, numConfs=num_conformers, params=AllChem.ETKDG())
  AllChem.MMFFOptimizeMoleculeConfs(mol)  # Corrected to optimize all conformers
  scores = []
  for cid in range(mol.GetNumConformers()):
      try:
        O3A = AllChem.GetO3A(mol, query, prbCid=cid)  # Get an O3A object for superposition
      except ValueError:
        return None
      # O3A.Align()
      alignment_score = O3A.Align()
      scores.append(alignment_score)
  return np.min(scores)


In [268]:
tqdm.pandas()

In [ ]:
# Add a new column 'Alignment_Score' to the DataFrame
dataset['ShapeSimilarity'] = dataset['rdkitmol'].progress_apply(lambda x: embed_optimize_align_molecule(x, query_h, num_conformers=2))

In [ ]:
dataset.sort_values(by='ShapeSimilarity', ascending=True)[:10]

In [ ]:
sns.relplot(x=np.arange(len(dataset)), y=dataset['ShapeSimilarity'].sort_values(ascending=False), aspect=2.0, height=2.0, kind='line')

In [ ]:
# Draw the first 10 molecules in a 2x5 grid
molecules_to_show = dataset.sort_values(by='ShapeSimilarity', ascending=True).iloc[:10]  # Select the first 10 rows
# Create a 2x5 grid of subplots for molecule display
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.subplots_adjust(hspace=0.5) # Adjust vertical spacing

# Loop through the subplots and display each molecule
for i, ax in enumerate(axes.ravel()):
    mol = molecules_to_show['rdkitmol'].iloc[i]  # Get the RDKit molecule
    chembl_id = molecules_to_show['chembl_id'].iloc[i]
    img = Draw.MolToImage(mol, size=(300, 300))  # Generate and display the image
    ax.imshow(img)  # Show the molecule image
    ax.set_title(chembl_id)  # Set a title for the subplot
    ax.axis('off')  # Turn off axis labels and ticks

plt.show()  # Display the figure with the molecules



**Exercise 4.1** Which are the molecules with worst-shape alignment? 

**Excesice 4.2** How is the relationship between Shapesimilarity and the other fingerprints? Are they correlated?

### RDKit and ADMET properties

<strong> Descriptor calculation

There are a lot of descriptors available in RDKit - http://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors

In [276]:
from rdkit.Chem import Descriptors

In [ ]:
Descriptors.TPSA(QueryH)

In [ ]:
Descriptors.MolLogP(QueryH)

<strong> Calculate molecular properties for Ro5 (Rule of five) of our top 5 compounds
    
 

In [ ]:
dataset.head(5)

In [282]:
#Define the descriptors to be analyzed
def calculate_ro5_properties(molecule):
    """
    Test if input molecule (SMILES) fulfills Lipinski's rule of five.

    """
    # RDKit molecule from SMILES
    # molecule = Chem.MolFromSmiles(smiles)
    # Calculate Ro5-relevant chemical properties
    molecular_weight = Descriptors.ExactMolWt(molecule)
    n_hba = Descriptors.NumHAcceptors(molecule)
    n_hbd = Descriptors.NumHDonors(molecule)
    logp = Descriptors.MolLogP(molecule)
    TPSA = Descriptors.TPSA (molecule)
    # Check if Ro5 conditions fulfilled
    conditions = [molecular_weight <= 500, n_hba <= 10, n_hbd <= 5, logp <= 5, TPSA < 140]
    ro5_fulfilled = sum(conditions) > 4
    # Return True if no more than one out of four conditions is violated
    return pd.Series(
        [molecular_weight, n_hba, n_hbd, logp, TPSA, ro5_fulfilled],
        index=['molecular_weight', 'n_hba', 'n_hbd', 'logp', 'TPSA', 'ro5_fulfilled'],
    )

In [ ]:
# Calculate RO5 properties
ro5_properties = dataset['rdkitmol'].apply(calculate_ro5_properties)
ro5_properties

In [ ]:
# Concatenate molecules with Ro5 data
MoleculeDatabase = pd.concat([dataset, ro5_properties], axis=1)
MoleculeDatabase

**Exercise 5** Use seaborn.displot and seaborn.pairplot to generate a histogram representation of the Lipinksi properties. Check the documentation online.

In [ ]:
# Note that the column "ro5_fulfilled" contains boolean values.
# Thus, we can use the column values directly to subset data.
# Note that ~ negates boolean values.
MoleculeDatabase_ro5_fulfilled = MoleculeDatabase[MoleculeDatabase['ro5_fulfilled']]
MoleculeDatabase_ro5_violated = MoleculeDatabase[~MoleculeDatabase['ro5_fulfilled']]

print(f'# compounds in unfiltered data set: {MoleculeDatabase.shape[0]}')
print(f'# compounds in filtered data set: {MoleculeDatabase_ro5_fulfilled.shape[0]}')
print(f'# compounds not compliant with the Ro5: {MoleculeDatabase_ro5_violated.shape[0]}')

In [ ]:
MoleculeDatabase_ro5_fulfilled

In [ ]:
# Save filtered data
MoleculeDatabase_ro5_fulfilled.to_csv('MoleculeDatabase_compounds_lipinski.csv')
MoleculeDatabase_ro5_fulfilled.head()

**Exercise 6**: Your budget only allows you to measure the activity of 5 molecules. Based on all the evidence generated in this notebook, which ones are you considering? Use rdkit to generate their formulas and CHEMBL API to retrieve attributes such as their name.